In [3]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection string
engine = create_engine('postgresql://uvg_user:uvg_password@db:5432/health_data')

# Simple query to test the foundation layer
try:
    df = pd.read_sql("SELECT 1 as connection_status", engine)
    print("Connection Successful! Your Biomedical Data Stack is ready.")
    print(df)
except Exception as e:
    print(f"Connection Failed: {e}")

Connection Successful! Your Biomedical Data Stack is ready.
   connection_status
0                  1


In [6]:
query = """
SELECT
  a.hadm_id,
  p.external_id,
  p.full_name,
  MAX(CASE WHEN d.label = 'White Blood Cells' THEN le.value_num END) AS max_wbc,
  MAX(CASE WHEN d.label = 'Lactate' THEN le.value_num END) AS max_lactate
FROM labevents le
JOIN d_labitems d ON d.labitem_id = le.labitem_id
JOIN admissions a ON a.hadm_id = le.hadm_id
JOIN patients p ON p.subject_id = a.subject_id
GROUP BY a.hadm_id, p.external_id, p.full_name
HAVING
  MAX(CASE WHEN d.label = 'White Blood Cells' THEN le.value_num END) > 12
  OR
  MAX(CASE WHEN d.label = 'Lactate' THEN le.value_num END) > 2;
"""
df = pd.read_sql(query, engine)
df


,hadm_id,external_id,full_name,max_wbc,max_lactate
0,10,MRN-0007,Miguel Castillo,13.0,NaN
1,6,MRN-0004,José Martínez,12.5,NaN
2,3,MRN-0001,Ana López,18.4,3.8
3,5,MRN-0003,María Gómez,14.2,2.2


In [4]:
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://uvg_user:uvg_password@db:5432/health_data"
)


In [7]:
import pandas as pd

pd.read_sql("""
SELECT
  p.subject_id,
  p.external_id,
  p.full_name,
  a.hadm_id,
  a.admission_type,
  a.admittime,
  a.dischtime,
  a.hospital_expire_flag
FROM patients p
JOIN admissions a ON a.subject_id = p.subject_id
ORDER BY a.admittime;
""", engine)


,subject_id,external_id,full_name,hadm_id,admission_type,admittime,dischtime,hospital_expire_flag
0,1,FINAL01,Paciente Final,1,Emergency,2101-01-10 08:00:00,2101-01-15 14:00:00,False
1,6,MRN-0005,Alex Rivera,7,Emergency,2101-02-05 09:30:00,2101-02-08 10:00:00,False
2,2,MRN-0001,Ana López,3,Emergency,2101-03-20 22:00:00,2101-03-28 10:00:00,True
3,7,MRN-0006,Lucía Herrera,9,Emergency,2101-05-14 07:00:00,2101-05-20 15:00:00,False
4,3,MRN-0002,Carlos Pérez,4,Urgent,2101-07-11 13:00:00,2101-07-14 11:00:00,False
5,8,MRN-0007,Miguel Castillo,10,Emergency,2101-08-21 20:00:00,2101-08-24 09:00:00,False
6,4,MRN-0003,María Gómez,5,Emergency,2101-09-02 06:00:00,2101-09-10 15:00:00,False
7,6,MRN-0005,Alex Rivera,8,Urgent,2101-11-01 16:00:00,2101-11-04 12:00:00,False
8,5,MRN-0004,José Martínez,6,Emergency,2101-12-18 19:00:00,2101-12-22 08:00:00,False
9,1,FINAL01,Paciente Final,2,Elective,2102-06-01 10:00:00,2102-06-05 09:00:00,False


In [8]:
pd.read_sql("""
SELECT
  p.external_id,
  p.full_name,
  COUNT(*) AS n_admissions
FROM patients p
JOIN admissions a ON a.subject_id = p.subject_id
GROUP BY p.subject_id, p.external_id, p.full_name
HAVING COUNT(*) > 1;
""", engine)


,external_id,full_name,n_admissions
0,MRN-0005,Alex Rivera,2
1,FINAL01,Paciente Final,2


In [9]:
pd.read_sql("""
SELECT
  a.hadm_id,
  p.external_id,
  p.full_name,
  EXTRACT(EPOCH FROM (a.dischtime - a.admittime)) / 86400.0
    AS length_of_stay_days
FROM admissions a
JOIN patients p ON p.subject_id = a.subject_id
ORDER BY length_of_stay_days DESC;
""", engine)


,hadm_id,external_id,full_name,length_of_stay_days
0,5,MRN-0003,María Gómez,8.375000
1,3,MRN-0001,Ana López,7.500000
2,9,MRN-0006,Lucía Herrera,6.333333
3,1,FINAL01,Paciente Final,5.250000
4,2,FINAL01,Paciente Final,3.958333
5,6,MRN-0004,José Martínez,3.541667
6,7,MRN-0005,Alex Rivera,3.020833
7,4,MRN-0002,Carlos Pérez,2.916667
8,8,MRN-0005,Alex Rivera,2.833333
9,10,MRN-0007,Miguel Castillo,2.541667


In [10]:
pd.read_sql("""
SELECT
  a.hadm_id,
  p.external_id,
  p.full_name,
  MAX(le.value_num) AS max_creatinine
FROM labevents le
JOIN d_labitems d ON d.labitem_id = le.labitem_id
JOIN admissions a ON a.hadm_id = le.hadm_id
JOIN patients p ON p.subject_id = a.subject_id
WHERE d.label = 'Creatinine'
GROUP BY a.hadm_id, p.external_id, p.full_name
ORDER BY max_creatinine DESC;
""", engine)


,hadm_id,external_id,full_name,max_creatinine
0,3,MRN-0001,Ana López,3.1
1,9,MRN-0006,Lucía Herrera,1.8
2,1,FINAL01,Paciente Final,1.6
3,8,MRN-0005,Alex Rivera,1.4
4,7,MRN-0005,Alex Rivera,0.9


In [11]:
pd.read_sql("""
SELECT
  a.hadm_id,
  p.external_id,
  p.full_name,
  MAX(le.value_num) AS max_creatinine
FROM labevents le
JOIN d_labitems d ON d.labitem_id = le.labitem_id
JOIN admissions a ON a.hadm_id = le.hadm_id
JOIN patients p ON p.subject_id = a.subject_id
WHERE d.label = 'Creatinine'
GROUP BY a.hadm_id, p.external_id, p.full_name
ORDER BY max_creatinine DESC;
""", engine)


,hadm_id,external_id,full_name,max_creatinine
0,3,MRN-0001,Ana López,3.1
1,9,MRN-0006,Lucía Herrera,1.8
2,1,FINAL01,Paciente Final,1.6
3,8,MRN-0005,Alex Rivera,1.4
4,7,MRN-0005,Alex Rivera,0.9


In [5]:
pd.read_sql("""
SELECT
  a.hadm_id,
  p.external_id,
  p.full_name,
  MAX(CASE WHEN d.label = 'White Blood Cells' THEN le.value_num END) AS max_wbc,
  MAX(CASE WHEN d.label = 'Lactate' THEN le.value_num END) AS max_lactate
FROM labevents le
JOIN d_labitems d ON d.labitem_id = le.labitem_id
JOIN admissions a ON a.hadm_id = le.hadm_id
JOIN patients p ON p.subject_id = a.subject_id
GROUP BY a.hadm_id, p.external_id, p.full_name
HAVING
  MAX(CASE WHEN d.label = 'White Blood Cells' THEN le.value_num END) > 12
  OR
  MAX(CASE WHEN d.label = 'Lactate' THEN le.value_num END) > 2;
""", engine)


,hadm_id,external_id,full_name,max_wbc,max_lactate
0,10,MRN-0007,Miguel Castillo,13.0,NaN
1,6,MRN-0004,José Martínez,12.5,NaN
2,3,MRN-0001,Ana López,18.4,3.8
3,5,MRN-0003,María Gómez,14.2,2.2
